In [44]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [4]:
cd '/content/drive/My Drive/Konkursy Kaggle/Predict Future Sales'

/content/drive/My Drive/Konkursy Kaggle/Predict Future Sales


In [71]:
train = pd.read_csv('/content/drive/My Drive/Konkursy Kaggle/Predict Future Sales/sales_train.csv').reset_index(drop=True)
test = pd.read_csv('/content/drive/My Drive/Konkursy Kaggle/Predict Future Sales/test.csv').reset_index(drop=True)
shops = pd.read_csv('/content/drive/My Drive/Konkursy Kaggle/Predict Future Sales/shops.csv').reset_index(drop=True)
items = pd.read_csv('/content/drive/My Drive/Konkursy Kaggle/Predict Future Sales/items.csv').reset_index(drop=True)
categories = pd.read_csv('/content/drive/My Drive/Konkursy Kaggle/Predict Future Sales/item_categories.csv').reset_index(drop=True)

## **Shops DB preprocessing**

In [72]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [ ]:
shops.shop_name.unique()

In [74]:
# extract the city name
shops["city"] = shops["shop_name"].apply(lambda s: s.split()[0])
shops["city_id"] = LabelEncoder().fit_transform(shops['city'])

#remove city name from shop name
f = lambda x: x["shop_name"].split(" ",1)[1]
shops["shop_name_clean"] = shops.apply(f, axis=1)


In [75]:
shops.head()

,shop_name,shop_id,city,city_id,shop_name_clean
0,"!Якутск Орджоникидзе, 56 фран",0,!Якутск,0,"Орджоникидзе, 56 фран"
1,"!Якутск ТЦ ""Центральный"" фран",1,!Якутск,0,"ТЦ ""Центральный"" фран"
2,"Адыгея ТЦ ""Мега""",2,Адыгея,1,"ТЦ ""Мега"""
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха,2,"ТРК ""Октябрь-Киномир"""
4,"Волжский ТЦ ""Волга Молл""",4,Волжский,3,"ТЦ ""Волга Молл"""


# **Category DB preprocessing**

In [93]:
categories.head()

,item_category_name,item_category_id,is_PS,is_XBOX
0,PC - Гарнитуры/Наушники,0,False,False
1,Аксессуары - PS2,1,True,False
2,Аксессуары - PS3,2,True,False
3,Аксессуары - PS4,3,True,False
4,Аксессуары - PSP,4,True,False


In [ ]:
categories.item_category_name.unique()

In [92]:
#add new feature - is_PS, is_XBOX
categories["is_PS"] = categories["item_category_name"].str.contains("- PS")
categories["is_XBOX"] = categories["item_category_name"].str.contains("XBOX")

In [24]:
df_all = pd.concat([train, test], sort=False)
if 'ID' in df_all: del df_all['ID']

In [28]:
df_all.Dates2 = pd.to_datetime(df_all.date)
df_all['year'] = df_all.Dates2.dt.year
df_all['month'] = df_all.Dates2.dt.month

df_all.loc[(df_all['month'] == 12) | (df_all['month'] == 1), 'Christmas_time'] = 1
df_all.loc[df_all['Christmas_time'].isnull(), 'Christmas_time'] = 0

df_all.loc[(df_all['item_cnt_day'] > 0) , 'is_not_canceled'] = 1
df_all.loc[df_all['is_not_canceled'].isnull(), 'is_not_canceled'] = 0

df_all['return_value_incl_returns'] = df_all['item_price']*df_all['is_not_canceled']*df_all['item_cnt_day']

df_all["cumsum"] = df_all["return_value_incl_returns"].cumsum()

In [29]:
df_all.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,year,month,Christmas_time,is_not_canceled,return_value_incl_returns,cumsum
0,02.01.2013,0.0,59,22154,999.00,1.0,2013.0,2.0,0.0,1.0,999.00,999.00
1,03.01.2013,0.0,25,2552,899.00,1.0,2013.0,3.0,0.0,1.0,899.00,1898.00
2,05.01.2013,0.0,25,2552,899.00,-1.0,2013.0,5.0,0.0,0.0,-0.00,1898.00
3,06.01.2013,0.0,25,2554,1709.05,1.0,2013.0,6.0,0.0,1.0,1709.05,3607.05
4,15.01.2013,0.0,25,2555,1099.00,1.0,2013.0,1.0,1.0,1.0,1099.00,4706.05


In [ ]:
df_all.item_cnt_day.value_counts()

In [34]:
#get top shops 20%
df_shops = (
    df_all[ ["return_value_incl_returns", "shop_id"] ]
    .groupby("shop_id")
    .agg("sum")
    .reset_index()
    .sort_values(by="return_value_incl_returns", ascending=False)
    .rename(columns={"return_value_incl_returns": "shop_price_total"})
)


df_shops["cumsum"] = df_shops["shop_price_total"].cumsum()
value_80prc = int(df_all["return_value_incl_returns"].sum() * 0.8)
df_shops["most_revenue_shops"] = df_shops["cumsum"] < value_80prc


top_shops = set(df_shops[ df_shops["most_revenue_shops"] ]["shop_id"].unique())
len(top_shops)

29